# News Category Classification using Word2Vec embeddings

University of Leeds

COMP5623 Artificial Intelligence

---

We will use two Python libraries:

1. **sklearn** a machine learning library

2. **Gensim** is a library for unsupervised topic modeling and natural language processing, using modern statistical machine learning

---


# 1. Dataset preparation - 20 Newsgroups

We will use sklearn to download **20 Newsgroups** (http://qwone.com/~jason/20Newsgroups/), a public available dataset of approximately 20,000 newsgroup posts, partitioned across 20 different newsgroups.  We will only load 3 categories for this example.

In [0]:
categories = [
      'comp.graphics',
      'sci.med',
      'rec.sport.baseball'
]

Load the subset of the 20 Newsgroups dataset.

In [0]:
from sklearn.datasets import fetch_20newsgroups

train_set = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
)

Look at some sample data.

In [32]:
train_set.target_names

['comp.graphics', 'rec.sport.baseball', 'sci.med']

In [33]:
print("Total number of news articles:", len(train_set.data))

Total number of news articles: 1775


In [34]:
print("\n".join(train_set.data[0].split("\n")[:10]))

From: geb@cs.pitt.edu (Gordon Banks)
Subject: Re: ORGAN DONATION AND TRANSPLANTATION FACT SHEET
Reply-To: geb@cs.pitt.edu (Gordon Banks)
Organization: Univ. of Pittsburgh Computer Science
Lines: 26

In article <1993Apr13.150018.641@lmpsbbs.comm.mot.com> dougb@ecs.comm.mot.com writes:

>My wife cannot donate blood because she has been to a malarial region
>in the past three years.  In fact, she tried to have her bone marrow


In [35]:
print(train_set.target_names[train_set.target[0]])

sci.med


# 2. Training a Word2Vec model

Now we will train a Word2Vec model which we will use to map each word in each news article to a feature representation. While the best models are trained on very large amounts of data, due to resources, we will use a model trained on this small corpus.

First we pre-process the data into lists of words.

In [0]:
parsed_train_data = []
for article in train_set.data:
  parsed_train_data.append(article.replace('\n',' ').split())

Train the model.

In [0]:
from gensim.models import Word2Vec

feature_length = 50

word2vec_model = Word2Vec(
    sentences=parsed_train_data,
    window=5,
    sg=1,       # Use skip-gram
    size=feature_length
)

How good is it? Sanity check...

In [86]:
word2vec_model.most_similar("disease", topn=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('common', 0.9158620834350586),
 ('evidence', 0.9105513095855713),
 ('cancer', 0.9032063484191895),
 ('treatment', 0.8932008147239685),
 ('therapies', 0.8925598859786987)]

In [87]:
word2vec_model.most_similar("baseball", topn=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('record', 0.9091603755950928),
 ('baseball.', 0.9035422205924988),
 ('fans', 0.895762026309967),
 ('weekend', 0.8953852653503418),
 ('division', 0.894350528717041)]

What is the feature representation for one word?

In [88]:
word2vec_model["baseball"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.37614304, -0.3212539 , -0.26331714,  0.34663174, -0.3686967 ,
        0.5305655 , -0.45642254, -0.19874948,  0.05603094, -0.02632546,
        0.8142622 ,  0.3124014 ,  0.55586684, -0.10897265, -0.34950596,
       -0.11198868,  0.28666997,  0.3239095 , -0.2045929 ,  0.32176694,
        0.05813552, -0.32818332, -0.17730637,  0.15533252,  0.07652529,
        0.03418873, -0.10511762, -0.36643463, -0.18065734, -0.2037365 ,
       -0.5100552 ,  0.03527943, -0.65899163,  0.2539918 ,  0.12292019,
        0.40873972, -0.4972573 ,  0.08840751,  0.288882  ,  0.27199146,
       -0.10281499,  0.01606054, -0.13584733,  0.17722136,  0.22349799,
        0.06968982,  0.01924053,  0.22805476, -0.4551319 , -0.18421626],
      dtype=float32)

# 3. Performing classification on news articles using feature embeddings

Finally, we will represent each news article as a block of word features, and perform classification on the embedded representations.

As an over-simplification of the problem (for the purposes of illustration), we will choose the N first words to represent an article so that all our article sizes are fixed.

In [0]:
article_size = 13 * feature_length

In [0]:
import numpy as np

def embed_article(article, cutoff):
  # Save the feature representation for each word in the article
  embedded_article = []
  for word in article:
    try:
      embedded_article.append(word2vec_model[word])
    except(KeyError): # Ignore words not in the model vocabulary
      pass
  return np.array(embedded_article).flatten()[:cutoff]

In [130]:
embedded_train_set = []

for article in parsed_train_data:
  embedded_train_set.append(embed_article(article, article_size))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Now we can try training a simple linear classifier to classify the news articles into their categories. 

In [0]:
from sklearn.linear_model import SGDClassifier

In [124]:
linear_classifier = SGDClassifier()

linear_classifier.fit(
      embedded_train_set,
      train_set.target,
)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

Can it predict on new articles?

In [139]:
new_articles = [
      # Article one - from latest NVidia news
      "Nvidia RTX 2080 Ti Cyberpunk 2077 GPU is revealed – but you can’t buy one." +
      "77 limited edition graphics cards are to be given away in a competition" +
      "The mysterious Cyberpunk 2077-themed GPU Nvidia recently teased has been revealed," +
      "and the reality of the graphics card may be a touch disappointing for some folks, in that" +
      "it isn’t a new model – and you won’t be able to buy one." +
      "The card is simply a GeForce RTX 2080 Ti (and appears to be exactly the same model," + 
      "and shroud design) decked out with the Cyberpunk 2077 colors and logo, which admittedly" +
      "looks pretty cool, but isn’t the GeForce RTX 2080 Ti Super" +
      "AMD confirms ‘Nvidia killer’ graphics card will be out in 2020",
      # Article two - from Health -> Oncology
      "Breast cancer test could predict chances of disease return 20 years later, study shows" +
      "Molecular nature of a woman’s breast cancer determines how their disease could progress," +
      "not just for the first five years, but also later,' says researcher" +
      "A new test could identify breast cancers that are likely to return more than 20 years later" +
      "development that might herald an era of personalised medicine." +
      "The way a patient’s cancer will progress can be determined by categorising molecular and" +
      "genetic markers of breast tumours into 11 subtypes, University of Cambridge researchers found." +  
      "Following around 2,000 women over 20 years, the team funded by the Cancer Research charity found" +
      "some women with initially aggressive cancers had a low chance of tumours returning after five years."
]

# Parse and embed
parsed_na = [a.replace('\n',' ').split() for a in new_articles]
embedded_new_articles = [embed_article(a, article_size) for a in parsed_na]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
predicted = linear_classifier.predict(embedded_new_articles)

In [143]:
for i, category in enumerate(predicted):
  print("New article", i, " predicted cateogry =>", train_set.target_names[category])

New article 0  predicted cateogry => comp.graphics
New article 1  predicted cateogry => sci.med
